In [2]:
import folium
import pandas as pd

In [3]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [15]:
import io, requests   # requests is optional

# --- simplest: let pandas read directly from the URL -----------------
csv_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
spacex_df = pd.read_csv(csv_url)

# quick look
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [16]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [17]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [18]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [19]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# -------------------------------------------------
# ❷  Add a marker + circle for each launch site
# -------------------------------------------------
for _, row in launch_sites_df.iterrows():
    site_name = row['Launch Site']
    lat, lon  = row['Lat'], row['Long']
    coordinate = [lat, lon]

    # coloured circle (1 km radius)
    folium.Circle(
        location=coordinate,
        radius=1000,                # metres
        color='#0000FF',            # blue outline
        fill=True,
        fill_opacity=0.2
    ).add_child(folium.Popup(site_name)).add_to(site_map)

    # text label
    folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(120, 12),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#0000FF;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

# -------------------------------------------------
# ❸  Display the interactive map
# -------------------------------------------------
site_map



In [20]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [21]:
marker_cluster = MarkerCluster()


In [22]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Colour code: green for success (class = 1), red for failure (class = 0)
spacex_df['marker_color'] = spacex_df['class'].apply(
    lambda x: 'green' if x == 1 else 'red'
)

# quick check
spacex_df[['Launch Site', 'class', 'marker_color']].head()


,Launch Site,class,marker_color
0,CCAFS LC-40,0,red
1,CCAFS LC-40,0,red
2,CCAFS LC-40,0,red
3,CCAFS LC-40,0,red
4,CCAFS LC-40,0,red


In [23]:
# Attach the cluster to the map
site_map.add_child(marker_cluster)

for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"{row['Launch Site']} — {'Success' if row['class']==1 else 'Failure'}",
        icon=folium.Icon(color=row['marker_color'], icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

site_map


In [24]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [25]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [26]:
# ------------------------------------------------------------
# ❶  Coordinates
# ------------------------------------------------------------
# launch‑site coordinates (taken from launch_sites_df)
site_name = 'CCAFS LC-40'
site_lat  = launch_sites_df.loc[launch_sites_df['Launch Site'] == site_name, 'Lat'].values[0]
site_lon  = launch_sites_df.loc[launch_sites_df['Launch Site'] == site_name, 'Long'].values[0]

# closest‑coastline point (←‑‑ replace these numbers with yours)
coast_lat, coast_lon = 28.56325, -80.56795          # example point on Florida shoreline

# ------------------------------------------------------------
# ❷  Great‑circle distance in kilometres
# ------------------------------------------------------------
distance_coastline = calculate_distance(site_lat, site_lon, coast_lat, coast_lon)
print(f"Distance {site_name} → coast: {distance_coastline:.2f} km")

# ------------------------------------------------------------
# ❸  Add markers + distance label onto the existing site_map
# ------------------------------------------------------------
# Marker at the coastline point
folium.Marker(
    location=[coast_lat, coast_lon],
    icon=folium.Icon(color='cadetblue', icon='flag', prefix='fa'),
    popup=f'Closest coast to {site_name}'
).add_to(site_map)

# Thin line connecting the two points
folium.PolyLine(
    locations=[[site_lat, site_lon], [coast_lat, coast_lon]],
    color='cadetblue',
    weight=2
).add_to(site_map)

# Text label showing the distance
distance_marker = folium.Marker(
    location=[(site_lat + coast_lat) / 2, (site_lon + coast_lon) / 2],  # midpoint
    icon=DivIcon(
        icon_size=(120, 12),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_coastline:.2f} km</b></div>'
    )
).add_to(site_map)

# Display the interactive map
site_map


Distance CCAFS LC-40 → coast: 0.92 km
